In [16]:
# Copyright (c) 2024, InfinityQ Technology Inc.

import copy
import pprint
import numpy as np
import logging
import json
from titanq import Model, Vtype, Target, S3Storage
import utils
import problem_gen

#### Setting Credentials

The user should configure their TitanQ API key here. For very large problems, the user must also configure an AWS Access key, AWS Secret Access key and AWS Bucket Name.

In [ ]:
logging.getLogger('botocore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)

# Enter your API Key Here
# Obtain your API key by contacting --> support@infinityq.tech
# Example: TITANQ_DEV_API_KEY = "00000000-0000-0000-0000-000000000000"
TITANQ_DEV_API_KEY = None

# Specify AWS keys and bucket name for solving very large problems
# AWS_ACCESS_KEY = "Your Access key"
# AWS_SECRET_ACCESS_KEY = "Your secret access key"
# AWS_BUCKET_NAME = "Your bucket name"

#### List of warehouses

In [18]:
with open("input/warehouses.json") as f:
    warehouses = json.load(f)

#### List of loans to be assigned to warehouses

In [19]:
with open("input/loans.json") as f:
    new_loans = json.load(f)

#### Generate dictionary of valid warehouse assignments for the loans

In [20]:
model = problem_gen.WarehouseLendingModel(new_loans,warehouses)

model.generate_valid_loan_warehouse_assignments()

#### Generate dictionary of loans that a warehouse could take (inverse of previous dictionary)

In [21]:
model.generate_valid_warehouse_loan_assignments()

#### Add variables to the model dictionary

In [22]:
model.add_variables()

#### Add constraints to the model dictionary

In [23]:
model.add_constraints()

#### Add objective function components

In [24]:
model.add_objective()

#### Set some hyperparameters for the strength of constraints

In [25]:
model.set_constraint_strength()

#### Compile model_dict to generate the input matrices to titanQ

In [26]:
num_variables, weight_matrix, bias_vector = utils.gen_data(model.model_dict)

#### Building the model on TitanQ

In [27]:
#############
# TitanQ SDK
#############
titanq_model = Model(
    api_key=TITANQ_DEV_API_KEY,
    # Insert storage_client parameter and specify corresponding AWS keys and bucket name for solving very large problems
    # storage_client=S3Storage(
    #     access_key=AWS_ACCESS_KEY,
    #     secret_key=AWS_SECRET_ACCESS_KEY,
    #     bucket_name=AWS_BUCKET_NAME
    # )
)
x = titanq_model.add_variable_vector('x', num_variables, Vtype.BINARY)
titanq_model.set_objective_matrices(weight_matrix, bias_vector, Target.MINIMIZE)

# Add constraints as expression
for con in model.model_dict["hard_constraints"]:
    expr = sum(var[1] * x[model.model_dict["name_id"][var[0][0]]] for var in con[0]) == 1
    titanq_model.add_constraint_from_expression(expr)

#### Setting TitanQ hyperparameters

In [28]:
num_chains = 32
num_engines = 16
T_min = 1.0
T_max = 1.0e6
temperatures = np.geomspace(T_min, T_max, num_chains, dtype=np.float32)
beta = (1.0/temperatures)
coupling_mult = 0.3
timeout_in_seconds = 3.0

#### Solving the model using TitanQ

In [29]:
response = titanq_model.optimize(
    beta = 1./temperatures,
    coupling_mult = coupling_mult, 
    timeout_in_secs = timeout_in_seconds, 
    num_chains = num_chains, 
    num_engines = num_engines
)

#### Extracting the solution

In [30]:
solution_found = False
best_objective = 0

best_idx = -1
best_objective = 0
best_assignment = []
best_wh_state = dict()

print("-------- CHECKING SOLUTIONS --------")
for idx, solution in enumerate(response.x):
    obj = np.dot(np.dot(solution, weight_matrix), solution) + np.dot(bias_vector, solution)
    passed = True
    loan_assignments = np.zeros(model.num_new_loans, dtype=int)

    # Extract loan assignments from solution state
    # Check set partitioning constraints
    for loan_i in range(model.num_new_loans):
        set_part_accum = 0

        # Check all warehouse assignment variables for loan_i
        # Exactly one of them should = 1, and the rest = 0
        for wh in model.valid_loan_wh[loan_i]:
            name = f'l_{loan_i}_wh_{wh}'

            var_id = model.model_dict["name_id"][name]

            if solution[var_id] == 1:
                set_part_accum += 1

                loan_assignments[loan_i] = wh
        
        if set_part_accum != 1:
            passed = False
            break
    
    if not passed:
        continue

    # Add loan values to the warehouses and pools that they're assigned to
    for i in range(model.num_new_loans):
        i_wh = loan_assignments[i] 

        warehouses[i_wh]["total_loans"] += new_loans[i]["value"]
        warehouses[i_wh]["available_funds"] -= new_loans[i]["value"]

        num_pools = len(warehouses[i_wh]["pools"])

        for k in range(num_pools):
            if warehouses[i_wh]["pools"][k]["name"] in new_loans[i]["pools"]:
                warehouses[i_wh]["pools"][k]["value"] += new_loans[i]["value"]


    # Update percentage values of pools in warehouses
    for j in range(model.num_warehouses):
        num_pools = len(warehouses[j]["pools"])

        for k in range(num_pools):
            warehouses[j]["pools"][k]["current_pct"] = warehouses[j]["pools"][k]["value"]/warehouses[j]["total_loans"]

    # Check if any pool limits are violated
    for wh in warehouses:
        for pool in wh["pools"]:
            if pool["current_pct"] > pool["limit_pct"]:
                passed = False
            
    
    if passed and obj < best_objective:
        best_assignment = loan_assignments.copy()
        
        best_wh_state = [copy.deepcopy(wh) for wh in warehouses]
        
        best_idx = idx
        best_objective = obj
        solution_found = True
        
    
if solution_found:
    print("Best Assignment: ", best_assignment)
    print("Best Solution Index: ", best_idx)
    print("")
    print("-------- Warehouse Dictionary After Loan Assignment --------")
    pprint.pprint(best_wh_state)

else:
    print("No valid solution reached")

-------- CHECKING SOLUTIONS --------
Best Assignment:  [1 0 1 0 1 1 1]
Best Solution Index:  0

-------- Warehouse Dictionary After Loan Assignment --------
[{'available_funds': 496.0,
  'name': 'small_bank_0',
  'pools': [{'category': 'location',
             'current_pct': 0.04807692307692308,
             'limit_pct': 0.1,
             'name': 'quebec',
             'value': 5.0},
            {'category': 'location',
             'current_pct': 0.09615384615384616,
             'limit_pct': 0.1,
             'name': 'ontario',
             'value': 10.0},
            {'category': 'location',
             'current_pct': 0.04807692307692308,
             'limit_pct': 0.075,
             'name': 'nova_scotia',
             'value': 5.0},
            {'category': 'purpose',
             'current_pct': 0.22115384615384615,
             'limit_pct': 0.25,
             'name': 'mortgage',
             'value': 23.0},
            {'category': 'purpose',
             'current_pct': 0.0480769